In [23]:
import pandas as pd
import boto3
from io import StringIO
from dateutil.parser import parse
from aws.mploader import LoadMP
from aws.s3_loader import load_from_s3, get_latest_keypath
from aws.s3 import create_s3_session
s3=create_s3_session()

In [24]:
date_str = get_latest_keypath()
date_param = parse(date_str).strftime("%Y/%m/%d")
# df = load_from_s3(date_str)
clean_df = LoadMP(date_str).fdf
# df

'2021/05/06'

In [31]:
def write_clean_df(clean_df):
    date_param = clean_df["Date"][0].strftime("%Y/%m/%d")
    csv_buffer = StringIO()
    clean_df.to_csv(csv_buffer)
    s3.put_object(
        Body=csv_buffer.getvalue(),
        Bucket="malaysia-stock-eod-data", 
        Key=f"clean_mplus/{date_param}/data.csv"
    )

In [32]:
keys = [
        obj["Key"]
        for obj in s3.list_objects(Bucket="malaysia-stock-eod-data")["Contents"]
        if obj["Key"].split("/")[0] == "mplus"
    ]
keys = [
        obj["Key"]
        for obj in s3.list_objects(Bucket="malaysia-stock-eod-data")["Contents"]
        if obj["Key"].split("/")[0] == "clean_mplus"
    ]
keys

['clean_mplus/2021/05/06/data.csv']

In [2]:
import os

# Get environment variables
# AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
# AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [3]:
# session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
# s3 = session.client("s3", region_name = "ap-southeast-1")
s3 = create_s3_session()
# response = s3.upload_file(
#     Filename = "C:/Users/jy/20210505_170708564037.csv",
#     Bucket="malaysia-stock-eod-data",
#     Key="mplus/2021/05/05/data.csv"
# )

In [4]:
keys = [obj["Key"] for obj in s3.list_objects(Bucket="malaysia-stock-eod-data")["Contents"]]

In [5]:
keys

['mplus/2021/05/05/data.csv', 'mplus/2021/05/06/data.csv']

In [75]:
response = s3.download_file(Filename = "./tempdata.csv", Bucket="malaysia-stock-eod-data", Key="mplus/2021/05/05/data.csv")

In [96]:
def parse_keypath(key):
    return "".join(key.split("/")[1:4])
parse_keypath(keys[0])

'20210505'

In [81]:
from dateutil.parser import parse


def load_from_s3(date_str):
    date_str = "2020-01-05"
    date_dt =  parse(date_str)
    date_param = date_dt.strftime("%Y/%m/%d")
    response = s3.get_object(Bucket="malaysia-stock-eod-data", Key=f"mplus/{date_param}/data.csv")
    csv_txt=response["Body"].read().decode('cp1256')
    df = pd.read_csv(StringIO(csv_txt), encoding="cp1256")
    return df


In [43]:
csv_txt=response["Body"].read().decode('cp1256')
csv_file = StringIO(csv_txt)
# df = pd.read_csv(csv_file, encoding="cp1256")

In [47]:
date = "20210505"
df = LoadMP(csv_file, date).fdf

In [64]:
cols = df.columns

In [49]:
from trade.load.mongo import initiate_mongo

In [51]:
client,coll = initiate_mongo()


In [60]:
cols2 = list(coll.find_one().keys())

In [61]:
len(cols2)

111

In [65]:
[j for j in cols if j not in cols2]


['RSSVol']